In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
from skimage.metrics import peak_signal_noise_ratio
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.optimizers import Adam
from google.colab.patches import cv2_imshow

def apply_median_filter(img):
    return cv2.medianBlur(img, 3)

def apply_mean_filter(img):
    return cv2.blur(img, (3, 3))

def apply_gaussian_filter(img):
    return cv2.GaussianBlur(img, (3, 3), 0)

def apply_bilateral_filter(img):
    return cv2.bilateralFilter(img, 5, 75, 75)

def get_psnr(original, processed):
    mse = np.mean((original - processed) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 10 * np.log10((max_pixel ** 2) / mse)
    return psnr

def add_poisson(img, lambda_val):
  poisson_noise = np.random.poisson(lambda_val, size=img.shape)
  noisy_image = img + poisson_noise
  if noisy_image.dtype != np.uint8:
    noisy_image = cv2.convertScaleAbs(noisy_image)
  return noisy_image

# Get the images in the pneumonia folder
data_folder = "/content/pneumonia/"
image_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.jpeg')]
images = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in image_files]

input_size = (124, 124)
images_resized = [cv2.resize(img, input_size) for img in images]
images_resized = np.array(images_resized)

train_images, test_images = train_test_split(images_resized, test_size=0.2, random_state=42)
train_images = tf.convert_to_tensor(train_images, dtype=tf.float32)
test_images = tf.convert_to_tensor(test_images, dtype=tf.float32)

# Autoencoder
def create_autoencoder(input_shape, z):
    input_img = Input(shape=input_shape)
    # Encoder
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(z, (3, 3), activation='relu', padding='same')(x)  # Use latent_dim here
    encoded = MaxPooling2D((2, 2), padding='same', name='bottleneck')(x)  # Adding a name to the bottleneck layer
    # Decoder
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    return autoencoder

lambda_values = [25, 50, 75]
noisy_images_list = []
for lambda_value in lambda_values:
    noisy_images = []
    for image in images_resized:
        noisy_image = add_poisson(image, lambda_value)
        noisy_images.append(noisy_image)
    noisy_images_list.append(noisy_images)

average_psnr_values = {}
input_shape = (124, 124, 1)
latent_dim = 32
for idx, lambda_value in enumerate(lambda_values):
    autoencoder = create_autoencoder(input_shape, latent_dim)  # Create autoencoder with specified input shape and latent dimension
    noisy_images = np.array(noisy_images_list[idx])
    autoencoder.fit(noisy_images, images_resized, epochs=100, batch_size=10, shuffle=True, validation_split=0.2)

    psnr_values_autoencoder = []
    psnr_values_median = []
    psnr_values_mean = []
    psnr_values_gaussian = []
    psnr_values_bilateral = []

    for test_image in test_images:
        denoised_image_autoencoder = autoencoder.predict(test_image[np.newaxis, ...])[0, ...]
        psnr_autoencoder = get_psnr(test_image, denoised_image_autoencoder)
        psnr_values_autoencoder.append(psnr_autoencoder)

        noisy_image = test_image.numpy()
        denoised_image_median = apply_median_filter(noisy_image)
        denoised_image_mean = apply_mean_filter(noisy_image)
        denoised_image_gaussian = apply_gaussian_filter(noisy_image)
        denoised_image_bilateral = apply_bilateral_filter(noisy_image)

        psnr_median = get_psnr(test_image, denoised_image_median)
        psnr_mean = get_psnr(test_image, denoised_image_mean)
        psnr_gaussian = get_psnr(test_image, denoised_image_gaussian)
        psnr_bilateral = get_psnr(test_image, denoised_image_bilateral)

        psnr_values_median.append(psnr_median)
        psnr_values_mean.append(psnr_mean)
        psnr_values_gaussian.append(psnr_gaussian)
        psnr_values_bilateral.append(psnr_bilateral)

    average_psnr_values[lambda_value] = {
        "Autoencoder": np.mean(psnr_values_autoencoder),
        "Median Filter": np.mean(psnr_values_median),
        "Mean Filter": np.mean(psnr_values_mean),
        "Gaussian Filter": np.mean(psnr_values_gaussian),
        "Bilateral Filter": np.mean(psnr_values_bilateral)
    }

# Print average PSNR values for each lambda value
for lambda_value, psnr_values in average_psnr_values.items():
    print(f"Lambda value: {lambda_value}")
    print(f"Average PSNR values:")
    for method, average_psnr in psnr_values.items():
        print(f"{method}: {average_psnr:.2f}")

Epoch 1/100
8/8 [==============================] - 3s 189ms/step - loss: -1065.6849 - val_loss: -1977.9160
Epoch 2/100
8/8 [==============================] - 1s 118ms/step - loss: -1880.2103 - val_loss: -1986.9363
Epoch 3/100
8/8 [==============================] - 1s 117ms/step - loss: -1882.0443 - val_loss: -1986.9421
Epoch 4/100
8/8 [==============================] - 1s 120ms/step - loss: -1882.0481 - val_loss: -1986.9420
Epoch 5/100
8/8 [==============================] - 1s 115ms/step - loss: -1882.0485 - val_loss: -1986.9420
Epoch 6/100
8/8 [==============================] - 1s 114ms/step - loss: -1882.0485 - val_loss: -1986.9420
Epoch 7/100
8/8 [==============================] - 1s 112ms/step - loss: -1882.0485 - val_loss: -1986.9420
Epoch 8/100
8/8 [==============================] - 1s 130ms/step - loss: -1882.0485 - val_loss: -1986.9420
Epoch 9/100
8/8 [==============================] - 1s 176ms/step - loss: -1882.0485 - val_loss: -1986.9420
Epoch 10/100
8/8 [===================

Lambda = 25 , Lambda = 50, Lambda = 75

Median Filter: 31.23, 31.23, 31.23

Mean Filter: 30.31, 30.31, 30.31

Bilateral Filter: 32.86, 32.86, 32.86

Gaussian Filter: 32.23, 32.23, 32.23

Autoencoder Model: 5.46, 5.46, 5.46

Answer the following questions:

a) Why is poisson distribution the ideal one to use to simulate noise for medical images? Why not gaussian or something else? Answer in terms of relevance of medical applications.

Medical images like X-rays and CT scans can be affected by specks of noise, making it harder to see important details. People can use "Poisson distribution" to understand this noise because it mimics how the image is created. This helps them develop better ways to remove the noise, like cleaning the specks off medical images, for better diagnosis and analysis. While other tools exist, they might not be as good at tackling this specific type of noise.

b) Which one performed the best? Why do you think this is the case?

The average PSNR values indicate that The Bilateral Filter is the best choice for denoising in medical imaging. This could be due to its ability to suppress noise while retaining critical structural details. Its effectiveness in maintaining edge sharpness and fine features results in better images. The filter's adaptability extends to various types and intensities of noise, including common Gaussian noise found in medical images, improving its utility across other imaging scenarios. Its customizable parameters allow for fine-tuning to address specific noise characteristics, offering flexibility and precision in denoising tasks.